# MIL - Multi instance Learning

In [11]:
import os
import datetime
import copy
import re
import yaml
import uuid
import warnings
import time
import inspect

import numpy as np
import pandas as pd
from functools import partial, reduce
from random import shuffle
import random

import torch
from torch import nn, optim
from torch import nn
from torch.nn import functional as F
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader
from torchvision.models import resnet
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import MNIST
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn import metrics as mtx
from sklearn import model_selection as ms
import os
import shutil
import warnings
import torch
import torch.nn as nn
import torchvision.models as models
from sklearn.preprocessing import StandardScaler
import numpy as np

warnings.filterwarnings("ignore")

In [12]:
from PIL import Image
from datetime import datetime
import random


def calculate_age(dob_str):
    formats = ["%m/%d/%Y", "%d-%m-%Y"]  # Two different format at the same time!
    for format_string in formats:
        try:
            dob_date = datetime.strptime(dob_str, format_string)
            current_date = datetime.now()
            age = (
                current_date.year
                - dob_date.year
                - (
                    (current_date.month, current_date.day)
                    < (dob_date.month, dob_date.day)
                )
            )
            return age
        except ValueError:
            continue


def encode_gender(gender):
    if gender == "F" or gender == "f":  # Two different gender value F and f ....
        return 0
    elif gender == "M" or gender == "m":
        return 1
    else:
        raise ValueError(
            "Invalid gender value. Expected 'F' or 'M', but received: {}".format(gender)
        )


class DLMICustomDataset(Dataset):
    def __init__(self, data, transform=None, flag="trainset", max_images=150):
        self.data = data
        self.transform = transform
        self.flag = flag
        self.max_images = max_images

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        patient_ID = self.data.iloc[idx, 1]
        img_path_folder = "data/raw/" + self.flag + "/" + str(patient_ID) + "/"

        images = []
        images_loaded = 0
        for filename in os.listdir(img_path_folder):
            if filename.endswith(".jpg"):
                img_path = os.path.join(img_path_folder, filename)
                image = Image.open(img_path).convert("RGB")
                if self.transform:
                    image = self.transform(image)
                images.append(image)
                images_loaded += 1
                if (
                    images_loaded >= self.max_images
                ):  # Stop loading if max_images reached
                    break
        if len(images) == 0:
            images.append(torch.zeros((3, 224, 224)))  # Placeholder image

        while len(images) < self.max_images:
            random_image = random.choice(images)
            images.append(random_image)

        label = torch.tensor(int(self.data.iloc[idx, 2]), dtype=torch.float)
        gender = torch.tensor(encode_gender(self.data.iloc[idx, 3]), dtype=torch.long)
        age = torch.tensor(calculate_age(self.data.iloc[idx, 4]), dtype=torch.float32)
        lymph_count = torch.tensor(self.data.iloc[idx, 5], dtype=torch.float32)
        clinical_data = torch.stack((gender, age, lymph_count))
        # (num_images, channels, height, width)
        return torch.stack(images), clinical_data, label

In [13]:
class HybridModel(nn.Module):
    def __init__(self, num_classes, mlp_input_dim, mlp_hidden_dim):
        super(HybridModel, self).__init__()
        self.resnet18 = models.resnet18(pretrained=True)
        num_ftrs = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Linear(num_ftrs, num_classes)
        self.mlp = MLP(
            input_dim=mlp_input_dim, hidden_dim=mlp_hidden_dim, output_dim=num_classes
        )
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear_classifier = nn.Linear(num_classes, 1)

    def forward(self, image_data, clinical_data):
        batch_size, num_images, channels, height, width = image_data.size()
        image_data = image_data.view(-1, channels, height, width)
        image_features = self.resnet18(image_data)
        image_features = image_features.view(batch_size, num_images, -1)
        image_features = self.pool(image_features)
        image_output = F.sigmoid(self.linear_classifier(image_features))
        clinical_output = F.sigmoid(self.mlp(clinical_data))
        combined_output = (image_output + clinical_output) / 2
        return combined_output


class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Training

In [35]:
import torchvision.transforms as transforms
from torchvision.datasets import DatasetFolder
from torch.utils.data import random_split


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = pd.read_csv(
    "/home/lujun/local/DLMI-Classification/data/raw/clinical_annotation.csv"
)

transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
custom_dataset = DLMICustomDataset(data=data, transform=transform)

train_size = int(0.8 * len(custom_dataset))
val_size = len(custom_dataset) - train_size
train_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)

model = HybridModel(num_classes=1, mlp_input_dim=3, mlp_hidden_dim=16).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, clinical, labels in train_loader:
        inputs, clinical, labels = (
            inputs.to(device),
            clinical.to(device),
            labels.to(device),
        )
        optimizer.zero_grad()
        outputs = model(inputs, clinical)
        labels = labels.unsqueeze(1)
        labels.requires_grad = True
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    # Validation
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, clinical, labels in val_loader:
            inputs, clinical, labels = (
                inputs.to(device),
                clinical.to(device),
                labels.to(device),
            )
            outputs = model(inputs, clinical)
            labels = labels.unsqueeze(1)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            total += labels.size(0)
            correct += (outputs == labels).sum().item()
    val_loss = val_loss / len(val_loader.dataset)  # Calculate validation loss
    val_accuracy = correct / total  # Calculate validation accuracy
    print(
        f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}"
    )

In [88]:
# Save the trained model
torch.save(model.state_dict(), "trained_model_original.pth")

In [152]:
# Evaluate the model on the test set and output predictions
model.eval()
all_predictions = []
test_data = pd.read_csv(
    "/home/lujun/local/DLMI-Classification/data/post-processed/clinical_annotation_test.csv"
)
test_dataset = DLMICustomDataset(
    data=test_data, transform=transform, flag="testset_flattened"
)

with torch.no_grad():
    all_predictions = []  # Initialize list to store predictions
    for i in range(len(test_dataset)):
        inputs, clinical, _ = test_dataset[i]  # Get inputs, clinical data, and label
        inputs, clinical = inputs.unsqueeze(0).to(device), clinical.unsqueeze(0).to(
            device
        )  # Add batch dimension and move to device
        image_outputs, mlp_outputs = model(inputs, clinical)  # Forward pass
        outputs = (image_outputs + mlp_outputs) / 2  # Average predictions
        _, predicted = torch.max(outputs, 1)  # Get predicted class
        all_predictions.append(predicted.item())  # Append predicted class to list

        # Print example outputs for debugging
        if i == 0:
            print("Example image outputs:", image_outputs)
            print("Example combined outputs:", outputs)

# Output all predictions
print("All Predictions:", all_predictions)

Example image outputs: tensor([[-0.7063,  0.5859]], device='cuda:0')
Example combined outputs: tensor([[-1.2863,  0.4067]], device='cuda:0')
All Predictions: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [5]:
data = pd.read_csv(
    "/home/lujun/local/DLMI-Classification/data/raw/clinical_annotation.csv"
)

In [7]:
data.LABEL.value_counts()  # Unbalanced data

 1    113
 0     50
-1     42
Name: LABEL, dtype: int64

In [139]:
df = pd.DataFrame({"Image_Name": images_names, "Prediction": all_predictions})

In [144]:
from collections import defaultdict

person_predictions = defaultdict(list)
images_names = []
source_folder_path = (
    "/home/lujun/local/DLMI-Classification/data/post-processed/testset_flattened"
)
for person_folder in os.listdir(source_folder_path):
    images_names.append(person_folder[:-4])

for filename, prediction in zip(images_names, all_predictions):
    person_number = filename.split("_")[0]  # Extract person number from filename
    person_predictions[person_number].append(prediction)

person_labels = {}
for person_number, predictions in person_predictions.items():
    count_0 = predictions.count(0)
    count_1 = predictions.count(1)
    print(f"This is the count: {count_0} + {count_1}")
    if count_0 > count_1:
        person_labels[person_number] = 0
    else:
        person_labels[person_number] = 1

# Output the final labels after majority voting
for person_number, label in person_labels.items():
    print(f"Person {person_number}: Predicted Label {label}")

This is the count: 0 + 41
This is the count: 0 + 69
This is the count: 0 + 92
This is the count: 0 + 103
This is the count: 0 + 51
This is the count: 0 + 137
This is the count: 0 + 58
This is the count: 0 + 112
This is the count: 0 + 85
This is the count: 0 + 166
This is the count: 0 + 50
This is the count: 0 + 46
This is the count: 0 + 76
This is the count: 0 + 59
This is the count: 0 + 52
This is the count: 0 + 145
This is the count: 0 + 167
This is the count: 0 + 134
This is the count: 0 + 88
This is the count: 0 + 142
This is the count: 0 + 164
This is the count: 0 + 42
This is the count: 0 + 63
This is the count: 0 + 69
This is the count: 0 + 47
This is the count: 0 + 180
This is the count: 0 + 44
This is the count: 0 + 71
This is the count: 0 + 38
This is the count: 0 + 115
This is the count: 0 + 40
This is the count: 0 + 33
This is the count: 0 + 45
This is the count: 0 + 42
This is the count: 0 + 53
This is the count: 0 + 50
This is the count: 0 + 91
This is the count: 0 + 51
T

In [142]:
len(all_predictions)

3258

In [103]:
images_names

['P59_000016',
 'P96_000063',
 'P23_000063',
 'P33_000109',
 'P0_000082',
 'P6_000006',
 'P135_000009',
 'P79_000005',
 'P90_000051',
 'P94_000003',
 'P40_000087',
 'P128_000090',
 'P101_000049',
 'P85_000038',
 'P112_000107',
 'P111_000174',
 'P168_000013',
 'P156_000014',
 'P28_000019',
 'P126_000096',
 'P166_000049',
 'P96_000026',
 'P113_000018',
 'P121_000039',
 'P123_000113',
 'P174_000052',
 'P30_000056',
 'P157_000033',
 'P102_000135',
 'P26_000046',
 'P79_000131',
 'P171_000011',
 'P43_000048',
 'P194_000073',
 'P198_000051',
 'P37_000023',
 'P35_000112',
 'P97_000054',
 'P55_000007',
 'P67_000080',
 'P165_000008',
 'P2_000056',
 'P112_000114',
 'P25_000009',
 'P15_000049',
 'P40_000086',
 'P94_000063',
 'P21_000007',
 'P134_000089',
 'P90_000018',
 'P37_000016',
 'P59_000161',
 'P168_000139',
 'P65_000075',
 'P162_000042',
 'P23_000107',
 'P126_000078',
 'P147_000143',
 'P44_000000',
 'P85_000024',
 'P136_000087',
 'P5_000097',
 'P134_000060',
 'P147_000095',
 'P137_000046',


In [93]:
test_dataset

Pretraining the resnet model

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import YourDataset  # 自定义数据集类，需根据实际情况修改

# 定义预处理转换
preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# 加载数据集
dataset = YourDataset(
    root="path/to/your/dataset", transform=preprocess
)  # 修改为你的数据集路径
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 加载预训练的ResNet模型
resnet = models.resnet18(pretrained=True)

# 冻结参数，只更新最后一层
for param in resnet.parameters():
    param.requires_grad = False

# 替换最后一层（全连接层）为自定义的嵌入层
num_features = resnet.fc.in_features
embedding_size = 128  # 嵌入向量的维度
resnet.fc = nn.Linear(num_features, embedding_size)

# 定义损失函数和优化器
criterion = nn.TripletMarginLoss()  # 使用三元组损失
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        # 正向传播
        embeddings = resnet(images)

        # 计算损失
        loss = criterion(embeddings, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(dataset)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# 保存模型
torch.save(resnet.state_dict(), "resnet18_embedding_model.pth")